# Ejercicio 1 - Experimentación Práctica

## Task 1 - Preparación del conjunto de datos

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# dataset Iris
iris = load_iris()
X = iris.data.astype(np.float32)
y = iris.target.astype(np.int64)

# entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# tensores de PyTorch
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_val_tensor = torch.tensor(X_val)
y_val_tensor = torch.tensor(y_val)

## Task 2 - Arquitectura modelo

In [ ]:
class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# parámetros del modelo
input_dim = X_train.shape[1]  # 4 features
hidden_dim = 16
output_dim = len(np.unique(y))  # 3 clases

# instanciar el modelo
model = FeedforwardNN(input_dim, hidden_dim, output_dim)
print(model)

FeedforwardNN(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=3, bias=True)
  (relu): ReLU()
)


## Task 3 - Funciones de Pérdida

In [3]:
def loss_function(loss_name):
    if loss_name == 'cross_entropy':
        return nn.CrossEntropyLoss()
    elif loss_name == 'mse':
        return nn.MSELoss()
    elif loss_name == 'nll':
        return nn.NLLLoss()
    else:
        raise ValueError('Función de pérdida no soportada')

def train_model(model, loss_fn, optimizer, X_train, y_train, X_val, y_val, epochs=50):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        if isinstance(loss_fn, nn.MSELoss):
            # One-hot para MSE
            y_train_oh = F.one_hot(y_train, num_classes=output_dim).float()
            loss = loss_fn(outputs, y_train_oh)
        elif isinstance(loss_fn, nn.NLLLoss):
            # LogSoftmax para NLLLoss
            outputs = F.log_softmax(outputs, dim=1)
            loss = loss_fn(outputs, y_train)
        else:
            loss = loss_fn(outputs, y_train)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

        # Validación
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            if isinstance(loss_fn, nn.MSELoss):
                y_val_oh = F.one_hot(y_val, num_classes=output_dim).float()
                val_loss = loss_fn(val_outputs, y_val_oh)
            elif isinstance(loss_fn, nn.NLLLoss):
                val_outputs = F.log_softmax(val_outputs, dim=1)
                val_loss = loss_fn(val_outputs, y_val)
            else:
                val_loss = loss_fn(val_outputs, y_val)
            val_losses.append(val_loss.item())
    return train_losses, val_losses

# uso con diferentes funciones de pérdida
loss_functions = ['cross_entropy', 'mse', 'nll']
results = {}
for loss_name in loss_functions:
    print(f'Entrenando con función de pérdida: {loss_name}')
    model = FeedforwardNN(input_dim, hidden_dim, output_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    loss_fn = loss_function(loss_name)
    train_losses, val_losses = train_model(model, loss_fn, optimizer, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, epochs=50)
    results[loss_name] = {'train_loss': train_losses, 'val_loss': val_losses}

Entrenando con función de pérdida: cross_entropy
Entrenando con función de pérdida: mse
Entrenando con función de pérdida: nll
Entrenando con función de pérdida: mse
Entrenando con función de pérdida: nll


## Task 4 - Técnicas de Regularización

In [ ]:
class FeedforwardNNDropout(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_p=0.5):
        super(FeedforwardNNDropout, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_p)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

def train_model_regularization(model, loss_fn, optimizer, X_train, y_train, X_val, y_val, epochs=50, l1_lambda=0.0):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        if isinstance(loss_fn, nn.MSELoss):
            y_train_oh = F.one_hot(y_train, num_classes=output_dim).float()
            loss = loss_fn(outputs, y_train_oh)
        elif isinstance(loss_fn, nn.NLLLoss):
            outputs = F.log_softmax(outputs, dim=1)
            loss = loss_fn(outputs, y_train)
        else:
            loss = loss_fn(outputs, y_train)
        # L1 regularization
        if l1_lambda > 0:
            l1_norm = sum(p.abs().sum() for p in model.parameters())
            loss = loss + l1_lambda * l1_norm
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

        # Validación
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            if isinstance(loss_fn, nn.MSELoss):
                y_val_oh = F.one_hot(y_val, num_classes=output_dim).float()
                val_loss = loss_fn(val_outputs, y_val_oh)
            elif isinstance(loss_fn, nn.NLLLoss):
                val_outputs = F.log_softmax(val_outputs, dim=1)
                val_loss = loss_fn(val_outputs, y_val)
            else:
                val_loss = loss_fn(val_outputs, y_val)
            if l1_lambda > 0:
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                val_loss = val_loss + l1_lambda * l1_norm
            val_losses.append(val_loss.item())
    return train_losses, val_losses

# diferentes técnicas de regularización
regularizations = {
    'none': {'dropout': False, 'l1': 0.0, 'l2': 0.0},
    'l2':   {'dropout': False, 'l1': 0.0, 'l2': 1e-3},
    'l1':   {'dropout': False, 'l1': 1e-3, 'l2': 0.0},
    'dropout': {'dropout': True, 'l1': 0.0, 'l2': 0.0}
}

results_reg = {}
for reg_name, reg_params in regularizations.items():
    print(f'Entrenando con regularización: {reg_name}')
    if reg_params['dropout']:
        model = FeedforwardNNDropout(input_dim, hidden_dim, output_dim, dropout_p=0.5)
    else:
        model = FeedforwardNN(input_dim, hidden_dim, output_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=reg_params['l2'])
    loss_fn = nn.CrossEntropyLoss()
    train_losses, val_losses = train_model_regularization(
        model, loss_fn, optimizer, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, epochs=50, l1_lambda=reg_params['l1']
    )
    results_reg[reg_name] = {'train_loss': train_losses, 'val_loss': val_losses}

Entrenando con regularización: none
Entrenando con regularización: l2
Entrenando con regularización: l1
Entrenando con regularización: l1
Entrenando con regularización: dropout
Entrenando con regularización: dropout
